In [19]:
import csv

class TobiiTsvObject:
    
    # Variables
    data = []
    headers = []
    
    #init
    def __init__(self, tsvPath):
        self.data, self.headers = self.importTsv(tsvPath)
    
    #Init Methods
    def importTsv(self, tsvPath):
        tobiiData = []
        with open(tsvPath,'rb') as tsvin:
            tsvin = csv.reader(tsvin, delimiter='\t')
            count = 0
            for row in tsvin:
                tobiiData.append(row)
        tobiiHeaders = []
        count = 0
        for header in tobiiData[0]:
            tobiiHeaders.append(str(header))
            print str(count) +": " + header
            count += 1
        return [tobiiData, tobiiHeaders]
    
    #Methods
    # call AOIHitCount(HeaderIndex) to get a count of hits misses and total within the AOI
    def AOIHitCount(self, HeaderIndex):
        hit = []
        for data in self.data[1:]:
            hit.append(data[HeaderIndex])
        hits = hit.count("1")
        miss = hit.count("0")
        total = hits + miss
        return [hits, miss, total]
    
    def getSaccades(self, AOIList, time=500):
        saccades = []            
        # 45 46 47            
        inputAOIs = AOIList
        prevAOI = None
        prevAOITime = 0
        currentAOI = None
        for row in tobii.data[1:]:
            for AOI in AOIList:
                if row[AOI] == "1":
                    prevAOI = currentAOI
                    currentAOI = tobii.headers[AOI]
                    if currentAOI == prevAOI:
                        prevAOITime = int(row[31])
                    if (int(row[31]) - prevAOITime) < time and currentAOI != prevAOI and prevAOI != None:
                        #print prevAOI[4:-4]+"\t->\t"+currentAOI[4:-4]
                        saccades.append([prevAOI[4:-4], currentAOI[4:-4], int(row[31]) - prevAOITime])#prevAOITime, int(row[31])])
        return saccades


In [21]:
tobii = TobiiTsvObject('ALL-DATA.tsv')

0: ﻿ExportDate
1: StudioVersionRec
2: StudioProjectName
3: StudioTestName
4: ParticipantName
5: [Q01]Value
6: [mq1]Value
7: [q2]Value
8: [q2_2]Value
9: [mq1_2]Value
10: [q2_1]Value
11: [mq1_1]Value
12: RecordingName
13: RecordingDate
14: RecordingDuration
15: RecordingResolution
16: PresentationSequence
17: FixationFilter
18: MediaName
19: MediaPosX (ADCSpx)
20: MediaPosY (ADCSpx)
21: MediaWidth
22: MediaHeight
23: SegmentName
24: SegmentStart
25: SegmentEnd
26: SegmentDuration
27: SceneName
28: SceneSegmentStart
29: SceneSegmentEnd
30: SceneSegmentDuration
31: RecordingTimestamp
32: LocalTimeStamp
33: EyeTrackerTimestamp
34: MouseEvent
35: FixationIndex
36: SaccadeIndex
37: GazeEventType
38: GazeEventDuration
39: FixationPointX (MCSpx)
40: FixationPointY (MCSpx)
41: SaccadicAmplitude
42: AbsoluteSaccadicDirection
43: RelativeSaccadicDirection
44: AOI[m1bMessy-AOI]Hit
45: AOI[m1bxnMessy]Hit
46: AOI[m1boMessy]Hit
47: AOI[m1bQuestion]Hit
48: AOI[m1aNotMesyQuestion]Hit
49: AOI[m1aNotMessy

In [22]:
tobii.AOIHitCount(48)

[788, 5588, 6376]

In [30]:
tobii.getSaccades([47,44])

[['m1bMessy-AOI', 'm1bQuestion', 96],
 ['m1bQuestion', 'm1bMessy-AOI', 90],
 ['m1bMessy-AOI', 'm1bQuestion', 94],
 ['m1bQuestion', 'm1bMessy-AOI', 107],
 ['m1bMessy-AOI', 'm1bQuestion', 167],
 ['m1bQuestion', 'm1bMessy-AOI', 160]]

In [176]:
#AOI seccades
index = 0
AOIIndex = []
for row in tobii.headers:
    if row[:3] == "AOI":
        AOIIndex.append(index)
    index += 1

seccades = []
    
prevAOI = None
currentAOI = None
for row in tobii.data[1:]:
    for AOIIdx in AOIIndex:
        if row[AOIIdx] == "1":
            currentAOI = tobii.headers[AOIIdx]
            if prevAOI != None and prevAOI != currentAOI:
                seccade = prevAOI+"->"+currentAOI
                seccades.append(seccade)
        else:
            currentAOI = None
    prevAOI = currentAOI
    
#print len(seccades)
            
saccades = []            
# 45 46 47            
inputAOIs = [45, 46, 47]
prevAOI = None
prevAOITime = 0
currentAOI = None
for row in tobii.data[1:]:
    for AOI in inputAOIs:
        if row[AOI] == "1":
            prevAOI = currentAOI
            currentAOI = tobii.headers[AOI]
            if currentAOI == prevAOI:
                prevAOITime = int(row[31])
            if (int(row[31]) - prevAOITime) < 500 and currentAOI != prevAOI and prevAOI != None:
                #print prevAOI[4:-4]+"\t->\t"+currentAOI[4:-4]
                saccades.append([prevAOI[4:-4], currentAOI[4:-4], prevAOITime, int(row[31])])
                

   


    